# 뉴스 요약하고 문서화하기

# API key 로딩

In [1]:
from dotenv import load_dotenv
import os

# .env 파일 불러오기
load_dotenv(override=True)

# 환경변수 가져오기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# print(OPENAI_API_KEY)

## 라이브러리 로딩 및 객체 생성

In [2]:
from openai import OpenAI
# openai api 인증 및 OpenAI 클라이언트(객체) 생성
client = OpenAI(api_key=OPENAI_API_KEY)

summary_prompt = """당신은 언어 이해 및 요약에 훈련된 고도로 숙련된 AI입니다.
다음 텍스트를 읽고 간결한 추상적 단락으로 요약했으면 합니다.
전체 텍스트를 읽을 필요 없이 토론의 요점을 이해하는 데 도움이 될 수 있는 일관되고
읽을 수 있는 요약을 제공하여 가장 중요한 요점을 유지하는 것을 목표로 합니다.
불필요한 세부 사항이나 접선 사항은 피하십시오."""
key_points_prompt = """당신은 정보를 핵심 포인트로 전달하는 데 특화된 능숙한 AI입니다.
다음 텍스트를 기반으로 논의되거나 언급된 주요 포인트를 확인하고 나열합니다.
이는 논의의 본질에 가장 중요한 아이디어, 결과 또는 주제가 되어야 합니다.
당신의 목표는 누군가가 읽을 수 있는 목록을 제공하여 이야기된 내용을 빠르게 이해하는 것입니다."""
action_items_prompt = """당신은 대화를 분석하고 행동 항목을 추출하는 데 있어 AI 전문가입니다.
본문을 검토하고 합의되거나 수행이 필요하다고 언급된 모든 작업, 과제 또는 행동을 식별하십시오.
이것들은 특정 개인에게 할당된 작업일 수도 있고 그룹이 취하기로 결정한 일반적인 행동일 수도 있습니다.
이러한 행동 항목을 명확하고 간결하게 나열하십시오."""
sentiment_prompt = """당신은 언어와 감정 분석에 전문성을 갖춘 AI로서 당신의 과제는 다음 텍스트의 감
정을 분석하는 것입니다.
토론의 전체적인 톤, 사용된 언어가 전달하는 감정, 단어와 구가 사용되는 맥락을 고려하십시오.
감정이 일반적으로 긍정적인지 부정적인지 중립적인지를 표시하고 가능한 한 당신의 분석에 대해 간략한 설명을 제
공하십시오."""

## 번역 함수

In [4]:
def text_extraction(text, prompt):
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        temperature=0,
        messages=[
        { "role": "system", "content": prompt },
        { "role": "user", "content": text }
        ]
    )
    return response.choices[0].message.content

### 음성 파일 -> text로 변환

In [7]:
audio_file_path = "datas/news.mp3"
with open(audio_file_path, 'rb') as audio_file:
    transcription = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file,
    response_format="text"
)

### summary 하기

In [18]:
# 요약용 프롬프트들 정의
# summary_prompt = "Summarize the transcription in 3 concise sentences."
# key_points_prompt = "Extract 3 to 5 key points from the transcription."
# action_items_prompt = "List the specific actions or next steps mentioned in the transcription."
# sentiment_prompt = "Analyze the overall sentiment (positive, neutral, or negative) of the transcription."


abstract_summary = text_extraction(transcription, summary_prompt)
key_points = text_extraction(transcription, key_points_prompt)
action_items = text_extraction(transcription, action_items_prompt)
sentiment = text_extraction(transcription, sentiment_prompt)

news_data = {
    'abstract_summary': abstract_summary,
    'key_points': key_points,
    'action_items': action_items,
    'sentiment': sentiment
}

for key, value in news_data.items():
    print(f'{key}\n {value}\n')



abstract_summary
 Due to a short rainy season and prolonged heatwave, wasp activity has increased significantly, leading to a surge in emergency calls to firefighters for nest removals. Firefighters, donned in protective white suits to avoid wasp attacks, are busier than ever, responding to up to 15 incidents a day during peak activity months of August and September. The increase in wasp-related emergencies has risen by 57% compared to the previous year, with firefighters advising the public to check for nests and maintain a distance of about 20 meters from active wasp sites to avoid attacks.

key_points
 1. 짧은 장마와 긴 폭염으로 인해 말벌 활동이 증가하였으며, 이로 인해 소방대원들의 벌집 제거 출동이 빈번해졌습니다.
2. 소방대원들은 보호복을 착용하고 스프레이를 사용하여 벌집을 제거하는 작업을 수행하고 있으며, 이 과정에서 말벌들이 흩어지거나 죽는 모습이 관찰됩니다.
3. 소방당국은 말벌의 공격을 피하기 위해 주변에 벌집이 있는지 확인하고 신고할 것을 권장하며, 말벌이 공격할 경우 즉시 현장에서 20미터 이상 떨어질 것을 조언합니다.
4. 지난달 소방에 접수된 벌집 제거 출동은 44,000건으로, 전년 대비 57% 증가하였습니다.

action_items
 1. 소방대원들이 보호복을 입고 스프레이를 사용하여 벌집을 제거합니다.
2. 소방대원들이 사람들을 대피시킨 후 벌집을 제거합

### doc 파일로 저장하기

In [21]:
import os
from docx import Document

# 폴더 자동 생성
os.makedirs("output", exist_ok=True)

doc = Document()

for key, value in news_data.items():
    # _로 나눈 단어들을 대문자로 만들고 공백으로 연결
    heading = ' '.join(word.capitalize() for word in key.split('_'))
    doc.add_heading(heading, level=1)
    doc.add_paragraph(value)
    # 섹션 사이에 줄바꿈 추가
    doc.add_paragraph()
    
# 워드 파일로 데이터 저장
doc.save('output/news_minutes.docx')